У цьому ДЗ ми потренуємось розв'язувати задачу багатокласової класифікації за допомогою логістичної регресії з використанням стратегій One-vs-Rest та One-vs-One, оцінити якість моделей та порівняти стратегії.

### Опис задачі і даних

**Контекст**

В цьому ДЗ ми працюємо з даними про сегментацію клієнтів.

Сегментація клієнтів – це практика поділу бази клієнтів на групи індивідів, які схожі між собою за певними критеріями, що мають значення для маркетингу, такими як вік, стать, інтереси та звички у витратах.

Компанії, які використовують сегментацію клієнтів, виходять з того, що кожен клієнт є унікальним і що їхні маркетингові зусилля будуть більш ефективними, якщо вони орієнтуватимуться на конкретні, менші групи зі зверненнями, які ці споживачі вважатимуть доречними та які спонукатимуть їх до купівлі. Компанії також сподіваються отримати глибше розуміння уподобань та потреб своїх клієнтів з метою виявлення того, що кожен сегмент цінує найбільше, щоб точніше адаптувати маркетингові матеріали до цього сегменту.

**Зміст**.

Автомобільна компанія планує вийти на нові ринки зі своїми існуючими продуктами (P1, P2, P3, P4 і P5). Після інтенсивного маркетингового дослідження вони дійшли висновку, що поведінка нового ринку схожа на їхній існуючий ринок.

На своєму існуючому ринку команда з продажу класифікувала всіх клієнтів на 4 сегменти (A, B, C, D). Потім вони здійснювали сегментовані звернення та комунікацію з різними сегментами клієнтів. Ця стратегія працювала для них надзвичайно добре. Вони планують використати ту саму стратегію на нових ринках і визначили 2627 нових потенційних клієнтів.

Ви маєте допомогти менеджеру передбачити правильну групу для нових клієнтів.

В цьому ДЗ використовуємо дані `customer_segmentation_train.csv`[скачати дані](https://drive.google.com/file/d/1VU1y2EwaHkVfr5RZ1U4MPWjeflAusK3w/view?usp=sharing). Це `train.csv`з цього [змагання](https://www.kaggle.com/datasets/abisheksudarshan/customer-segmentation/data?select=train.csv)

**Завдання 1.** Завантажте та підготуйте датасет до аналізу. Виконайте обробку пропущених значень та необхідне кодування категоріальних ознак. Розбийте на тренувальну і тестувальну вибірку, де в тесті 20%. Памʼятаємо, що весь препроцесинг ліпше все ж тренувати на тренувальній вибірці і на тестувальній лише використовувати вже натреновані трансформери.
Але в даному випадку оскільки значень в категоріях небагато, можна зробити обробку і на оригінальних даних, а потім розбити - це простіше. Можна також реалізувати процесинг і тренування моделі з пайплайнами. Обирайте як вам зручніше.

In [131]:
import pandas as pd
import numpy as np
df = pd.read_csv('customer_segmentation_train.csv')

In [132]:
df.head()

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,Segmentation
0,462809,Male,No,22,No,Healthcare,1.0,Low,4.0,Cat_4,D
1,462643,Female,Yes,38,Yes,Engineer,NaN,Average,3.0,Cat_4,A
2,466315,Female,Yes,67,Yes,Engineer,1.0,Low,1.0,Cat_6,B
3,461735,Male,Yes,67,Yes,Lawyer,0.0,High,2.0,Cat_6,B
4,462669,Female,Yes,40,Yes,Entertainment,NaN,High,6.0,Cat_6,A


In [133]:
df.describe()

,ID,Age,Work_Experience,Family_Size
count,8068.000000,8068.000000,7239.000000,7733.000000
mean,463479.214551,43.466906,2.641663,2.850123
std,2595.381232,16.711696,3.406763,1.531413
min,458982.000000,18.000000,0.000000,1.000000
25%,461240.750000,30.000000,0.000000,2.000000
50%,463472.500000,40.000000,1.000000,3.000000
75%,465744.250000,53.000000,4.000000,4.000000
max,467974.000000,89.000000,14.000000,9.000000


In [134]:
df.dtypes

ID                   int64
Gender              object
Ever_Married        object
Age                  int64
Graduated           object
Profession          object
Work_Experience    float64
Spending_Score      object
Family_Size        float64
Var_1               object
Segmentation        object
dtype: object

In [135]:
print(df.isnull().sum(), '\n')
print(np.round(100 * df.isnull().sum() / df.shape[0], 2))

ID                   0
Gender               0
Ever_Married       140
Age                  0
Graduated           78
Profession         124
Work_Experience    829
Spending_Score       0
Family_Size        335
Var_1               76
Segmentation         0
dtype: int64 

ID                  0.00
Gender              0.00
Ever_Married        1.74
Age                 0.00
Graduated           0.97
Profession          1.54
Work_Experience    10.28
Spending_Score      0.00
Family_Size         4.15
Var_1               0.94
Segmentation        0.00
dtype: float64


Пропущених значень в змінних `Var_1` та `Graduated` менше 1%, тому просто видалимо ці порожні значення. По колонці `Ever_Married`, `Profession`, `Work_Experience`, `Family_Size` здійснимо імпутацію для кожної з змінних за окремими логіками:
- Ever_Married: для **null** значень заповнимо їх 0, так як таких значень менше 2%, це не сильно повпливає на результат вибірки і це інтуїтивно ближче до правди
- Profession: таких записів 124, присвоїмо їх до мажорного класу Artist
- Work_Experience: записів 829, припускаємо, що **null** значення описують таких, що не мають досвіду, тобто їхній досвід складає 0, заповнимо null 0
- Family_Size: нул записів 335, заповнимо їх значенням 1, так як мінімальна можлива кількість членів родини - 1

In [136]:
df.drop(df[df['Var_1'].isnull()].index.values, inplace=True)
df.drop(df[df['Graduated'].isnull()].index.values, inplace=True)

In [137]:
df['Ever_Married'].fillna(0, inplace=True)
df['Profession'].fillna('Artist', inplace=True)
df['Work_Experience'].fillna(0, inplace=True)
df['Family_Size'].fillna(1, inplace=True)

In [138]:
df.isnull().sum()

ID                 0
Gender             0
Ever_Married       0
Age                0
Graduated          0
Profession         0
Work_Experience    0
Spending_Score     0
Family_Size        0
Var_1              0
Segmentation       0
dtype: int64

In [139]:
for column in df.select_dtypes('object').columns:
    print(f"Unique values for {column} column {df[column].unique()}\n")

Unique values for Gender column ['Male' 'Female']

Unique values for Ever_Married column ['No' 'Yes' 0]

Unique values for Graduated column ['No' 'Yes']

Unique values for Profession column ['Healthcare' 'Engineer' 'Lawyer' 'Entertainment' 'Artist' 'Executive'
 'Doctor' 'Homemaker' 'Marketing']

Unique values for Spending_Score column ['Low' 'Average' 'High']

Unique values for Var_1 column ['Cat_4' 'Cat_6' 'Cat_7' 'Cat_3' 'Cat_1' 'Cat_2' 'Cat_5']

Unique values for Segmentation column ['D' 'A' 'B' 'C']



In [140]:
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder

gender_map = {"Male": 0, "Female": 1}
ever_marrier_map = {"No": 0, "Yes": 1}
graduated_map = {"No": 0, "Yes": 1}
segmentation_map = {"A": 0, "B": 1, "C": 2, "D": 3}

df['Gender'] = df['Gender'].replace(gender_map)
df['Ever_Married'] = df['Ever_Married'].replace(ever_marrier_map)
df['Graduated'] = df['Graduated'].replace(graduated_map)
df['Segmentation'] = df['Segmentation'].replace(segmentation_map)

spending_core_encoder = OrdinalEncoder(categories=[['Low', 'Average', 'High']])
spending_core_encoder.fit(df[['Spending_Score']])
df['Spending_Score'] = spending_core_encoder.transform(df[['Spending_Score']])


categorical_columns = ['Profession', 'Var_1']
one_hot_encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
one_hot_encoder.fit(df[categorical_columns])
encoded_categorical_columns = list(one_hot_encoder.get_feature_names_out())
df[encoded_categorical_columns] = one_hot_encoder.transform(df[categorical_columns])


In [141]:
df.drop(columns=categorical_columns, inplace=True)

In [142]:
from sklearn.model_selection import train_test_split

X = df.loc[:, df.columns != 'Segmentation']
y = df['Segmentation']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

**Завдання 2. Важливо уважно прочитати все формулювання цього завдання до кінця!**

Застосуйте методи ресемплингу даних SMOTE та SMOTE-Tomek з бібліотеки imbalanced-learn до тренувальної вибірки. В результаті у Вас має вийти 2 тренувальних набори: з апсемплингом зі SMOTE, та з ресамплингом з SMOTE-Tomek.

Увага! В нашому наборі даних є як категоріальні дані, так і звичайні числові. Базовий SMOTE не буде правильно працювати з категоріальними даними, але є його модифікація, яка буде. Тому в цього завдання є 2 виконання

  1. Застосувати SMOTE базовий лише на НЕкатегоріальних ознаках.

  2. Переглянути інформацію про метод [SMOTENC](https://imbalanced-learn.org/dev/references/generated/imblearn.over_sampling.SMOTENC.html#imblearn.over_sampling.SMOTENC) і використати цей метод в цій задачі. За цей спосіб буде +3 бали за це завдання і він рекомендований для виконання.

  **Підказка**: аби скористатись SMOTENC треба створити змінну, яка містить індекси ознак, які є категоріальними (їх номер серед колонок) і передати при ініціації екземпляра класу `SMOTENC(..., categorical_features=cat_feature_indeces)`.
  
  Ви також можете розглянути варіант використання варіації SMOTE, який працює ЛИШЕ з категоріальними ознаками [SMOTEN](https://imbalanced-learn.org/dev/references/generated/imblearn.over_sampling.SMOTEN.html)

In [143]:
from imblearn.over_sampling import SMOTE, SMOTENC
from imblearn.combine import SMOTETomek

In [144]:
smote = SMOTE(random_state=42)
X_train_res_smote, y_train_res_smote = smote.fit_resample(X_train, y_train)

smote_tomek = SMOTETomek(random_state=42)
X_train_res_smote_tomek, y_train_res_smote_tomek = smote_tomek.fit_resample(X_train, y_train)

In [145]:
print('Original train dataset: ', X_train.shape)
print('Resampled train dataset with SMOTE: ', X_train_res_smote.shape)
print('Resampled train dataset with SMOTETomek: ', X_train_res_smote_tomek.shape)

Original train dataset:  (6331, 24)
Resampled train dataset with SMOTE:  (7116, 24)
Resampled train dataset with SMOTETomek:  (5216, 24)


In [146]:
smotenc = SMOTENC(categorical_features=[1, 2, 4, 6])
X_train_res_smotenc, y_train_res_smotenc = smotenc.fit_resample(X_train, y_train)

In [147]:
print('Resampled train dataset with SMOTENC: ', X_train_res_smotenc.shape)
print('\nOriginal y values:', y_train_res_smotenc.value_counts())
print('\nResampled y values with SMOTENC:', y_train.value_counts())

Resampled train dataset with SMOTENC:  (7116, 24)

Original y values: Segmentation
2    1779
0    1779
3    1779
1    1779
Name: count, dtype: int64

Resampled y values with SMOTENC: Segmentation
3    1779
2    1547
0    1543
1    1462
Name: count, dtype: int64


**Завдання 3**.
  1. Навчіть модель логістичної регресії з використанням стратегії One-vs-Rest з логістичною регресією на оригінальних даних, збалансованих з SMOTE, збалансованих з Smote-Tomek.  
  2. Виміряйте якість кожної з натренованих моделей використовуючи `sklearn.metrics.classification_report`.
  3. Напишіть, яку метрику ви обрали для порівняння моделей.
  4. Яка модель найкраща?
  5. Якщо немає суттєвої різниці між моделями - напишіть свою гіпотезу, чому?

In [154]:
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier, OneVsOneClassifier
from sklearn.metrics import classification_report

# One-vs-Rest
log_reg = LogisticRegression(solver='liblinear')
ovr_model = OneVsRestClassifier(log_reg)
ovr_model.fit(X_train, y_train)
ovr_predictions = ovr_model.predict(X_test)

# Обчислимо метрики precision та recall для кожного класу на оригінальних даних
print("Classification report for original data")
print(classification_report(y_test, ovr_predictions))

ovr_model = OneVsRestClassifier(log_reg)
ovr_model.fit(X_train_res_smote, y_train_res_smote)
ovr_predictions = ovr_model.predict(X_test)

# Обчислимо метрики precision та recall для кожного класу з resample SMOTE
print("Classification report for SMOTE resampled data")
print(classification_report(y_test, ovr_predictions))

ovr_model = OneVsRestClassifier(log_reg)
ovr_model.fit(X_train_res_smote_tomek, y_train_res_smote_tomek)
ovr_predictions = ovr_model.predict(X_test)

# Обчислимо метрики precision та recall для кожного класу з resample SMOTE
print("Classification report for SMOTETomek resampled data")
print(classification_report(y_test, ovr_predictions))

Classification report for original data
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       386
           1       0.00      0.00      0.00       365
           2       0.00      0.00      0.00       387
           3       0.28      1.00      0.44       445

    accuracy                           0.28      1583
   macro avg       0.07      0.25      0.11      1583
weighted avg       0.08      0.28      0.12      1583

Classification report for SMOTE resampled data
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       386
           1       0.23      1.00      0.37       365
           2       0.00      0.00      0.00       387
           3       0.00      0.00      0.00       445

    accuracy                           0.23      1583
   macro avg       0.06      0.25      0.09      1583
weighted avg       0.05      0.23      0.09      1583

Classification report for SMOTETomek resamp

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  

In [157]:
# One-vs-One
log_reg = LogisticRegression(solver='liblinear')
ovr_model = OneVsOneClassifier(log_reg)
ovr_model.fit(X_train, y_train)
ovr_predictions = ovr_model.predict(X_test)

# Обчислимо метрики precision та recall для кожного класу на оригінальних даних
print("Classification report for original data")
print(classification_report(y_test, ovr_predictions))

ovr_model = OneVsOneClassifier(log_reg)
ovr_model.fit(X_train_res_smote, y_train_res_smote)
ovr_predictions = ovr_model.predict(X_test)

# Обчислимо метрики precision та recall для кожного класу з resample SMOTE
print("Classification report for SMOTE resampled data")
print(classification_report(y_test, ovr_predictions))

ovr_model = OneVsOneClassifier(log_reg)
ovr_model.fit(X_train_res_smote_tomek, y_train_res_smote_tomek)
ovr_predictions = ovr_model.predict(X_test)

# Обчислимо метрики precision та recall для кожного класу з resample SMOTE
print("Classification report for SMOTETomek resampled data")
print(classification_report(y_test, ovr_predictions))

ovr_model = OneVsOneClassifier(log_reg)
ovr_model.fit(X_train_res_smotenc, y_train_res_smotenc)
ovr_predictions = ovr_model.predict(X_test)

# Обчислимо метрики precision та recall для кожного класу з resample SMOTE
print("Classification report for SMOTENC resampled data")
print(classification_report(y_test, ovr_predictions))

Classification report for original data
              precision    recall  f1-score   support

           0       0.38      0.25      0.31       386
           1       0.21      0.02      0.04       365
           2       0.46      0.58      0.51       387
           3       0.44      0.79      0.56       445

    accuracy                           0.43      1583
   macro avg       0.37      0.41      0.36      1583
weighted avg       0.38      0.43      0.37      1583

Classification report for SMOTE resampled data
              precision    recall  f1-score   support

           0       0.46      0.47      0.47       386
           1       0.40      0.28      0.33       365
           2       0.48      0.61      0.54       387
           3       0.68      0.67      0.68       445

    accuracy                           0.52      1583
   macro avg       0.51      0.51      0.50      1583
weighted avg       0.51      0.52      0.51      1583

Classification report for SMOTETomek resamp

Стратегія `One-vs-Rest` показала гірші результати, аніж стратегія `One-vs-One` у поєднанні з логістичною регресією. Щодо ресемплінгу, SMOTENC та SMOTE показали найкращі результати. Залежно від поставленої задачі, я обрав метрику precision, оскільки неправильне передбачення не має явної критичної точки некоректного прогнозу. Якби це була задача виявлення хвороби - я обрав би recall, так як нам важливо не помилитись у визначення діагнозу. Так як це задача передбачення для маркетингу, ми можемо дозволити помилитись, проте в даному випадку важливіже "потрапити в ціль" та обрати коректних клієнтів. Отже, стратегія `One-vs-One` у поєднанні з `SMOTENC` дала найкращі, проте далеко не ідеальні результати. Macro avg - 0.51.